# Applied Data Science Capstone
## Assignments
This notebook would be available on Github under the following link (Github handle @mukulbiswas)

In [2]:
import pandas as pd
import numpy as np

print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Scraping and Preparation of Toronto Neighbourhood Data\
## Content of this notebook
The notebook scrapes a suggested Wikipedia pages for Toronto neighbourhood details in it. It contains the postcode details in a tabular format. Following steps are required as per the assignment given -

Broad approach:
1. Scrape the table from the Wikipedia page
2. Preprocess the data in using pandas


_*Using the notebook from an earlier assignment.*_

In [2]:
import pandas as pd
import numpy as np
import requests as req

In [4]:
wikipedia_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [7]:
from lxml import html

myWikiPage = req.get(wikipedia_link)
page = myWikiPage.text

# Load the page (string) in a DOM structure
dom_html = html.fromstring(page)

# Use XPath to find the first instance of <title> as a DOM element
dom_rows = dom_html.xpath('/html/body/div[3]/div[3]/div[4]/div/table[1]/tbody/tr')

print('No of rows in the table are', len(dom_rows))

No of rows in the table are 290


_*290 rows including the header. There are 289 data rows.*_

In [23]:
# Create a dataframe with the required columns
data_columns = ['PostCode','Borough','Neighbourhood']
df = pd.DataFrame(columns=data_columns)

rowcount = len(dom_rows)

# for all the rows, iterate. Skip the header row.
for x1 in range(1, rowcount):
    tr_node = dom_rows[x1]
    new_row = {}

    # for each column, iterate
    for y1 in range(0,3):
        # if the content is text, store in the dict
        if (type(tr_node[y1].text) is str):
            new_row[data_columns[y1]] = tr_node[y1].text.strip()
        
        # else if the content is another tag (anchor-tag), then read the inner text
        else:
            new_row[data_columns[y1]] = tr_node[y1][0].text.strip()
    
    # insert the dict of 3 items into the df as a new row. x1 is the index
    df.loc[x1] = new_row

print('No of rows processed and columns are', df.shape)
df.head()

No of rows processed and columns are (289, 3)


,PostCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


Remove the rows where Borough is *Not assigned*.

In [24]:
# Remove all the items where both borough and neighbourhood are "Not assigned"
df_2 = df[(df['Borough']!='Not assigned')]

print('The new shape of the dataframe is', df_2.shape)
df_2.head()

The new shape of the dataframe is (212, 3)


,PostCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


77 rows are dropped.

In [25]:
# Find the indexes of the rows where the neighbourhood are not assigned

indexes = df_2[df_2['Neighbourhood']=='Not assigned'].index.values

# For those indexes, replace the name of the neighbourhood with that of the Borough
df_2.loc[indexes,'Neighbourhood'] = df_2.loc[indexes,'Borough']

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [27]:
# Check if there is any row left with neighbourhood == 'Not assigned'

len(df_2[df_2['Neighbourhood']=='Not assigned'])

0

In [29]:
# Merge rows of same Post Codes; with comma-separated Borough names
df_3 = pd.DataFrame({'Neighborhood' : df_2.groupby([ 'PostCode','Borough'])['Neighbourhood'].apply(','.join)})

print('The new shape of the dataframe is', df_3.shape)
df_3.head(10)

The new shape of the dataframe is (103, 1)


,,Neighborhood
PostCode,Borough,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"


## Summary and Assumptions
The above dataframe contains the scraped and preprocessed data from Wikipedia that is indexed by PostCode with multiple neighbourhood names captured against it.

It is assumed while processing the data that -
1. The term "Not assigned" has the fixed pattern. A variation with addition whitespaces would not result the same way.
2. There are no blank cells in the wikipedia page. If existed, it should have been replaced with "Not assigned" for pre-processing.
3. The wikipedia page format does not changes. The XPath of the data-table has been obtained using a browser feature availabile in developer-mode.

The final shape of the dataframe is captured below-

In [30]:
print('The final shape of the dataframe is', df_3.shape)

The final shape of the dataframe is (103, 1)
